In [ ]:
!git clone https://github.com/qiuqiangkong/audioset_tagging_cnn
!pip install librosa soundfile wget torchlibrosa

%cd audioset_tagging_cnn/pytorch

Cloning into 'audioset_tagging_cnn'...
remote: Enumerating objects: 714, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 714 (delta 131), reused 120 (delta 120), pack-reused 560 (from 2)
Receiving objects: 100% (714/714), 3.99 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (412/412), done.
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=eb908c9b517d03839144c3e2893caf41b21c490bde83cd613da69ea15fd53956
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
/content/audioset_tagging_cnn/pytorch


In [ ]:
import glob
import librosa
import numpy as np
import os
import soundfile as sf
import pandas as pd
import torch
import torchaudio

from collections import defaultdict
from pathlib import Path
from pytorch_utils import move_data_to_device
from models import Cnn14
from models import Cnn14_DecisionLevelMax
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Cnn14_DecisionLevelMax(sample_rate=32000, window_size=1024, hop_size=320, mel_bins=64, fmin=50, fmax=14000, classes_num=527)

#model = Cnn1.
#4(sample_rate=32000, window_size=1024, hop_size=320, mel_bins=64, fmin=50, fmax=14000,
              #classes_num=527)

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/bsc_thesis/Cnn14_mAP=0.431.pth', map_location=device)
model.load_state_dict(checkpoint['model'])
model.to(device)
model.eval()

Cnn14_DecisionLevelMax(
  (spectrogram_extractor): Spectrogram(
    (stft): STFT(
      (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
      (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
    )
  )
  (logmel_extractor): LogmelFilterBank()
  (spec_augmenter): SpecAugmentation(
    (time_dropper): DropStripes()
    (freq_dropper): DropStripes()
  )
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), b

In [ ]:
# Load label classes
with open('../metadata/class_labels_indices.csv') as f:
    lines = f.readlines()[1:]
    idx_to_label = [line.strip().split(',')[2].strip('"') for line in lines]

In [ ]:
# Labelling of one file
audio_path = '/content/drive/MyDrive/bsc_thesis/early_media_sample/1.wav'
(audio, sr) = librosa.load(audio_path, sr=32000, mono=True)

audio_tensor = torch.tensor(audio).float().unsqueeze(0)  # (1, time_steps)
audio_tensor = move_data_to_device(audio_tensor, device)

with torch.no_grad():
    output = model(audio_tensor, None)
    clipwise_output = output['clipwise_output'].data.cpu().numpy()[0]

In [ ]:
category_mapping = {
    "Speech": [
        "speech", "talking", "conversation", "narration", "monologue", "dialogue", "discussion",
        "babbling", "shout", "whispering", "chanting", "yell", "laugh", "cry", "bellow",
        "whoop", "scream", "giggle", "snicker", "chuckle", "whimper", "moan", "sigh",
        "mantra", "groan", "grunt", "breathing", "wheeze", "snooring", "gasp", "pant",
        "snort", "cough", "throat clearing", "sneeze", "sniff", "chewing", "biting", "gargling",
        "hiccup", "cheering", "speech", "cough"
    ],
    "Silence": [
        "silence", "quiet", "ambient", "run", "static", "vibration"
    ],
    "Music": [
        "music", "singing", "instrument", "orchestra", "piano", "guitar", "choir", "song",
        "humming", "melody", "drum", "beat", "rhythm", "synthesizer", "electronic",
        "yodeling", "chant", "rapping", "whistling", "applause", "banjo",
        "sitar", "mandolin", "zither", "ukulele", "piano", "organ", "sampler", "harpsichord",
        "percussion", "timpani", "tabla", "cymbal", "hi-hat", "wood block", "tambourine",
        "maraca", "gong", "mallet", "marimba", "glockenspiel", "vibraphone", "steelpan",
        "orchestra", "horn", "trumpet", "trombone", "string", "violin", "pizzicato", "cello", "bass",
        "flute", "saxophone", "clarinet", "harp", "tuning fork", "chime", "harmonica",
        "accordion", "bagpipes", "didgeridoo", "shofar", "theremin", "scratching",
        "heavy metal", "rock", "grunge", "rhythm", "reggae", "country", "bluegrass", "funk",
        "jazz", "disco", "opera", "techno", "dubstep", "electronica", "flamenco", "blues",
        "capella", "ska", "lullaby"
    ],
    "Ringing": [
        "ring", "bell", "phone", "alarm", "chime", "beep", "notification", "timer", "noise",
        "pulse", "sine wave", "heartbeat", "telephone", "dial", "siren", "buzzer", "tick", "ding"
    ]
}

def categorize_label(display_name):
    display_name_lower = display_name.lower()

    for category, keywords in category_mapping.items():
        if any(keyword in display_name_lower for keyword in keywords):
            return category

    return "Unknown" # Values with unknown category should later be removed

In [ ]:
top_k = 20
top_indices = np.argsort(clipwise_output)[::-1][:top_k]
for idx in top_indices:
  category = categorize_label(idx_to_label[idx])
  print(f"{category:<30}: {clipwise_output[idx]:.3f}")
  #print(f"{idx_to_label[idx]:<30}: {clipwise_output[idx]:.3f}")

Music                         : 0.898
Speech                        : 0.810
Music                         : 0.382
Music                         : 0.342
Music                         : 0.333
Music                         : 0.319
Music                         : 0.225
Speech                        : 0.213
Music                         : 0.210
Music                         : 0.189
Music                         : 0.151
Music                         : 0.137
Music                         : 0.136
Speech                        : 0.131
Unknown                       : 0.130
Music                         : 0.119
Unknown                       : 0.104
Music                         : 0.101
Speech                        : 0.101
Music                         : 0.089


In [ ]:
AUDIO_DIR = "/content/drive/MyDrive/bsc_thesis/early_media_sample"
SAMPLE_RATE = 32000
HOP_SIZE = 320           # ~10ms at 32kHz
FRAMES_PER_SECOND = SAMPLE_RATE // HOP_SIZE  # = 100 frames/sec

# Get list of .wav files in the folder
wav_files = glob.glob(os.path.join(AUDIO_DIR, '*.wav'))

print(wav_files)

['/content/drive/MyDrive/bsc_thesis/early_media_sample/893.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/499.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/2919.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/4345.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/443.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/85.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/1047.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/394.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/2591.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/4060.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/2083.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/1722.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/965.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/1614.wav', '/content/drive/MyDrive/bsc_thesis/early_media_sample/772.wav', '/content/drive/MyDrive/bsc_thes

In [ ]:
# Batch labelling for the whole dataset
index_to_category = {}
for idx, label in enumerate(idx_to_label):
    mapped = categorize_label(label)
    if mapped != "Unknown":
        index_to_category[idx] = mapped

target_categories = ["Speech", "Music", "Ringing", "Silence"]

# --- Process each audio file ---
results = []
audio_files = [f for f in os.listdir(AUDIO_DIR) if f.endswith('.wav') and len(f) < 7]

for fname in tqdm(audio_files, desc="Extracting normalized soft targets"):
    fpath = os.path.join(AUDIO_DIR, fname)

    try:
        waveform, _ = librosa.load(fpath, sr=SAMPLE_RATE, mono=True)
        x = torch.tensor(waveform).float().unsqueeze(0)
        x = move_data_to_device(x, device)

        with torch.no_grad():
            output = model(x, None)
            print(f"Output keys for {fname}: {output.keys()}")
            frame_probs = output['framewise_output'].data.cpu().numpy()[0]  # shape: (T, 527)

        total_frames = frame_probs.shape[0]
        num_seconds = total_frames // FRAMES_PER_SECOND

        for sec in range(num_seconds):
          start_idx = sec * FRAMES_PER_SECOND
          end_idx = start_idx + FRAMES_PER_SECOND
          second_slice = frame_probs[start_idx:end_idx]  # shape: (100, 527)

          # For each frame, determine top predicted class
          top_classes = np.argmax(second_slice, axis=1)

          # Map those to categories
          mapped_categories = [categorize_label(idx_to_label[c]) for c in top_classes]

          if len(set(mapped_categories)) != 1:
              continue  # if mixed predictions then skip this second

          category = mapped_categories[0]
          if category == "Unknown":
              continue

          # Aggregate soft targets from all frames
          category_probs = defaultdict(float)
          for frame in second_slice:
              for idx, prob in enumerate(frame):
                  cat = index_to_category.get(idx, None)
                  if cat:
                      category_probs[cat] += prob

          soft_vector = np.array([
              category_probs.get("Speech", 0.0),
              category_probs.get("Music", 0.0),
              category_probs.get("Ringing", 0.0),
              category_probs.get("Silence", 0.0)
          ])
          total = np.sum(soft_vector)
          if total == 0:
              continue
          normalized_soft = soft_vector / total

          row = {
              "file": fname,
              "start_time": sec,
              "hard_target": category,
              "soft_speech": normalized_soft[0],
              "soft_music": normalized_soft[1],
              "soft_ringing": normalized_soft[2],
              "soft_silence": normalized_soft[3],
          }
          results.append(row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")


Extracting normalized soft targets:   1%|          | 1/99 [00:00<01:31,  1.07it/s]

Error processing 85.wav: Given input size: (512x1x8). Calculated output size: (512x0x4). Output size is too small


Extracting normalized soft targets:   2%|▏         | 2/99 [00:01<01:28,  1.09it/s]

Error processing 51.wav: Expected 2D or 3D (batch mode) tensor with possibly 0 batch size and other non-zero dimensions for input, but got: [1, 1, 0]
Output keys for 99.wav: dict_keys(['framewise_output', 'clipwise_output'])


Extracting normalized soft targets:   3%|▎         | 3/99 [00:03<01:44,  1.08s/it]

...

Extracting normalized soft targets:  98%|█████████▊| 97/99 [03:15<00:00,  2.44it/s]

Output keys for 8.wav: dict_keys(['framewise_output', 'clipwise_output'])


Extracting normalized soft targets:  99%|█████████▉| 98/99 [03:15<00:00,  2.73it/s]

Output keys for 7.wav: dict_keys(['framewise_output', 'clipwise_output'])


Extracting normalized soft targets: 100%|██████████| 99/99 [03:15<00:00,  1.98s/it]


In [ ]:
OUTPUT_CSV = "/content/audio_labels.csv"

# --- Save to CSV ---
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved results to: {OUTPUT_CSV}")

print(df.to_string())


Saved results to: /content/audio_labels.csv
        file  start_time hard_target  soft_speech  soft_music  soft_ringing  soft_silence
0     99.wav           1      Speech     0.901278    0.066291      0.027042      0.005390
1     99.wav           2      Speech     0.891663    0.072630      0.029005      0.006702
2     99.wav           3      Speech     0.876059    0.081586      0.034869      0.007486
3     99.wav           4      Speech     0.850595    0.105968      0.037115      0.006322
4     99.wav           5      Speech     0.852581    0.107647      0.031733      0.008038
5     99.wav           6      Speech     0.808446    0.114271      0.053223      0.024060
6     99.wav           8      Speech     0.876242    0.086865      0.020217      0.016675
7     99.wav           9      Speech     0.868596    0.103775      0.021943      0.005685
8     99.wav          10      Speech     0.869055    0.097107      0.025669      0.008169
9     99.wav          11      Speech     0.872097    0.

In [ ]:
# Encode class labels
le = LabelEncoder()
df["label_encoded"] = le.fit_transform(df["hard_target"])

# Feature extractor: log-mel spectrogram
def extract_logmel_features(file_path, sr=32000, n_mels=64, duration=1.0):
    try:
        y, _ = librosa.load(file_path, sr=sr, duration=duration)
        if len(y) < int(sr * duration):
            y = np.pad(y, (0, int(sr * duration) - len(y)))
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        logmel_spec = librosa.power_to_db(mel_spec)
        return logmel_spec.mean(axis=1)  # Average over time axis
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Split ensuring segments from the same clip go to the same set
unique_clips = df["file"].unique()
train_clips, test_clips = train_test_split(
    unique_clips,
    test_size=0.2,
    stratify=df.groupby("file")["hard_target"].first(),
    random_state=42
)

train_df = df[df["file"].isin(train_clips)]
test_df = df[df["file"].isin(test_clips)]

# Extract features for a DataFrame
def extract_features_and_labels(df_subset):
    features, labels = [], []
    for _, row in df_subset.iterrows():
        feat = extract_logmel_features(os.path.join(AUDIO_DIR, row["file"]))
        if feat is not None:
            features.append(feat)
            labels.append(row["label_encoded"])
    return np.array(features), np.array(labels)

# Extract train/test features
X_train, y_train = extract_features_and_labels(train_df)
X_test, y_test = extract_features_and_labels(test_df)

In [ ]:
# Train Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Classification Report:
              precision    recall  f1-score   support

       Music       0.77      0.97      0.86        62
     Ringing       0.65      0.94      0.77        80
     Silence       0.71      0.48      0.57        25
      Speech       0.88      0.51      0.65       102

    accuracy                           0.74       269
   macro avg       0.75      0.72      0.71       269
weighted avg       0.77      0.74      0.72       269



In [ ]:
import joblib

# Save model and label encoder
joblib.dump(clf, "rf_student_model.pkl")
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']

In [ ]:
# Inference function
def predict(filepath):
    feat = extract_logmel_features(filepath)
    if feat is None:
        return None
    feat = feat.reshape(1, -1)
    pred_encoded = clf.predict(feat)[0]
    return le.inverse_transform([pred_encoded])[0]

# Example usage
test_files = [os.path.join(AUDIO_DIR, "100.wav"),
              os.path.join(AUDIO_DIR, "101.wav"),
              os.path.join(AUDIO_DIR, "102.wav"),
              os.path.join(AUDIO_DIR, "102.wav"),
              os.path.join(AUDIO_DIR, "110.wav"),
              os.path.join(AUDIO_DIR, "111.wav")]  # Replace with your actual files

for file in test_files:
    pred = predict(file)
    print(f"{file}: {pred}")

/content/drive/MyDrive/bsc_thesis/early_media_sample/100.wav: Speech
/content/drive/MyDrive/bsc_thesis/early_media_sample/101.wav: Speech
/content/drive/MyDrive/bsc_thesis/early_media_sample/102.wav: Speech
/content/drive/MyDrive/bsc_thesis/early_media_sample/102.wav: Speech
/content/drive/MyDrive/bsc_thesis/early_media_sample/110.wav: Music
/content/drive/MyDrive/bsc_thesis/early_media_sample/111.wav: Music


## Failed tests

In [ ]:
from sklearn.model_selection import train_test_split

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["label_encoded"] = le.fit_transform(df["hard_target"])

# Get unique clip IDs
unique_clips = df["file"].unique()
train_clips, test_clips = train_test_split(unique_clips, test_size=0.2, random_state=42, stratify=df.groupby("file")["hard_target"].first())

# Split DataFrame
train_df = df[df["file"].isin(train_clips)]
test_df = df[df["file"].isin(test_clips)]

# Paper uses mel spectogram as features, soft targets are used as features in here
# Features and labels
X_train = train_df[["soft_speech", "soft_music", "soft_ringing", "soft_silence"]].values
y_train = train_df["label_encoded"].values
X_test = test_df[["soft_speech", "soft_music", "soft_ringing", "soft_silence"]].values
y_test = test_df["label_encoded"].values


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib  # For saving the model

# Train Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Save model and label encoder for future use
joblib.dump(clf, "student_random_forest_model.pkl")
joblib.dump(le, "label_encoder.pkl")


Classification Report:
              precision    recall  f1-score   support

       Music       1.00      1.00      1.00        62
     Ringing       1.00      1.00      1.00        80
     Silence       1.00      1.00      1.00        25
      Speech       1.00      1.00      1.00       102

    accuracy                           1.00       269
   macro avg       1.00      1.00      1.00       269
weighted avg       1.00      1.00      1.00       269



['label_encoder.pkl']

In [ ]:
# Load model and label encoder
clf = joblib.load("student_random_forest_model.pkl")
le = joblib.load("label_encoder.pkl")

# Predict on new data
X_new = [[0.85, 0.1, 0.02, 0.03]]  # example soft targets
predicted_class = le.inverse_transform(clf.predict(X_new))[0]
print("Predicted label:", predicted_class)

Predicted label: Speech


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load CSV with per-second labeled data
df = pd.read_csv("/content/audio_labels.csv")

# Encode labels (e.g., speech → 0, music → 1, ...)
le = LabelEncoder()
df["label_encoded"] = le.fit_transform(df["hard_target"])

# Features: soft targets from teacher
X = df[["soft_speech", "soft_music", "soft_ringing", "soft_silence"]].values

# Target: hard labels
y = df["label_encoded"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Parameters — simple and fast to train
params = {
    "objective": "multiclass",
    "num_class": 4,
    "metric": "multi_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "num_leaves": 31,
}

# Train with early stopping via callback
bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    num_boost_round=100,
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# Evaluate
y_pred = bst.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=le.classes_))


Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.000522728

Classification Report:
              precision    recall  f1-score   support

       Music       1.00      1.00      1.00       117
     Ringing       1.00      1.00      1.00       112
     Silence       1.00      1.00      1.00        67
      Speech       1.00      1.00      1.00       323

    accuracy                           1.00       619
   macro avg       1.00      1.00      1.00       619
weighted avg       1.00      1.00      1.00       619

